**Regression**



In [2]:
import numpy as np

from keras.layers import Dense, Input
from keras.models import Model, load_model
from keras.optimizers import Adam

Using TensorFlow backend.


**Introducing new Libraries**

* **pandas**: Library offering data structures and operation for data manipulation and analysis.

* **sklearn**: Library providing machine learning algorithms and tools.

In [6]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.utils import shuffle

**The Data**

We are going to be using a dataset about wine quality. The input variables will give us information about the wine like pH and alcohol percentage. The output variable correlates these inputs to the wine quality.

This time, the data is not well organized and is inside a csv file. We are going to read this file and prepare the data for training.

In [9]:
# Loading the dataset file
import os.path
path = "Datasets/winequalityN.csv"
if os.path.isfile("../" + path) :
    dataset = pd.read_csv("../" + path)
else:
    dataset = pd.read_csv("2-Basic-neural-networks/" + path)
    
# Let's take a look at the columns we have in this dataset.
print("Dataset Columns: {}\n".format(dataset.columns.values))

Dataset Columns: ['type' 'fixed acidity' 'volatile acidity' 'citric acid' 'residual sugar'
 'chlorides' 'free sulfur dioxide' 'total sulfur dioxide' 'density' 'pH'
 'sulphates' 'alcohol' 'quality']



**Removing invalid rows**

Sometimes, the datasets might be incomplete. This is the case of this dataset and we will remove rows with missing values.

In [10]:
# Removes rows with invalid values
n_rows_b4 = dataset.shape[0]
dataset = dataset.dropna(how='any',axis=0)
n_rows_c = dataset.shape[0]
print("{} rows containing invalid data removed.".format((n_rows_b4-n_rows_c)))

34 rows containing invalid data removed.


**Shuffling the dataset**

One way to improve the training performance, is to shuffle the dataset before training.

When the model learns, it overseers patterns and tries to correlate those patterns to the output. If the occurence of these patterns are not evenly distributed throughout the dataset, the network might focus only on the patterns that occurs the most. Also, we might have missing inputs or outputs values missing when we separate it into train and test data if the dataset if completely sequentially organized.

In [6]:
# Shuffles the dataset
dataset = shuffle(dataset)

**Separating into input data and output data**

Right now, we have the entire dataset inside the same array. We need to separate it so we can tell our model what are the inputs and outputs. 

In [7]:
# Separate into input and outputs of the network
predictors = dataset.iloc[:,0:12].values
wine_quality = dataset.iloc[:,12].values.astype(np.float32)

**Categorical Encoding**

The first column of our dataset is "type". It can be either "white" or "red". The network doesn't understand that. It only understands numbers.

We could attribute each type with a number. Let's red is 0, and white is 1. But that would not work! The network needs a clearer distinction of what is what.

Instead, we are going to give each wine time its own neuron. If it is "red", only the first input neuron will be 1. If it "white", only the second neuron will be 1.

**TODO: The teacher needs to explain the importance of categorical encoding.**

In [8]:
# Encodes categorized values
print("Cathegory before encoding (10 first): {}".format(predictors[0:10,0]))

ct = ColumnTransformer([("type", OneHotEncoder(),[0])], remainder="passthrough")
predictors = ct.fit_transform(predictors)
print("Cathegory after encoding (10 first):\n{}".format(predictors[0:10,0:2]))

Cathegory before encoding (10 first): ['white' 'white' 'white' 'red' 'white' 'white' 'white' 'red' 'white'
 'white']
Cathegory after encoding (10 first):
[[0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [1.0 0.0]
 [0.0 1.0]
 [0.0 1.0]
 [0.0 1.0]
 [1.0 0.0]
 [0.0 1.0]
 [0.0 1.0]]


**Separating into train and test datasets**

_We are almost there!_

We removed invalid data, shuffled, and encoded the dataset.
Now we can finally separate our data to train and test.

In [20]:
train_ratio = 0.8
train_index = int(train_ratio*predictors.shape[0])
print("Total: {0}. Train: {1}. Test: {2}".format(predictors.shape[0], train_index, predictors.shape[0]-train_index))

x_train = predictors[0:train_index]
x_train = x_train.astype(np.float32)
y_train = wine_quality[0:train_index]

x_test = predictors[train_index:predictors.shape[0]]
x_test = x_test.astype(np.float32)
y_test = wine_quality[train_index:predictors.shape[0]]

Total: 6463. Train: 5170. Test: 1293


In [21]:
print("Input training data shape:", x_train.shape)
print("Example input training data:",x_train[0])
print("Output training data shape:", y_train.shape)
print("Example output training data:",y_train[0])

print("\nInput test data shape:", x_test.shape)
print("Output test data shape:", y_test.shape)

Input training data shape: (5170, 13)
Example input training data: [0.0 1.0 6.8 0.23 0.29 12.2 0.035 38.0 236.0 0.9976 3.35 0.52 9.8]
Output training data shape: (5170,)
Example output training data: 6.0

Input test data shape: (1293, 13)
Output test data shape: (1293,)


**Building a regression model**

It's your time to build a model, train, test, and save it.

In [38]:
def build_model():
    # Input layer
    inputs = Input([13, ], name='data')
    # Hidden layers
    model = Dense(32, activation='relu')(inputs)
    model = Dense(48, activation='relu')(model)
    model = Dense(48, activation='relu')(model)
    model = Dense(32, activation='relu')(model)
    # Output layer
    outputs = Dense(1, activation='linear', name='fc3')(model)

    # Define the model
    model = Model(inputs=[inputs], outputs=outputs)

    model.compile(optimizer=Adam(lr=2e-5), #Gradient Descend Algorithm.
                  loss='mse',       #MSE = Mean Squared Error
                  metrics=['mae'])  #MAE = Mean Absolute Error
    return model

net = build_model()
print(net.summary())

Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
data (InputLayer)            (None, 13)                0         
_________________________________________________________________
dense_18 (Dense)             (None, 32)                448       
_________________________________________________________________
dense_19 (Dense)             (None, 48)                1584      
_________________________________________________________________
dense_20 (Dense)             (None, 48)                2352      
_________________________________________________________________
dense_21 (Dense)             (None, 32)                1568      
_________________________________________________________________
fc3 (Dense)                  (None, 1)                 33        
Total params: 5,985
Trainable params: 5,985
Non-trainable params: 0
_________________________________________________________

In [39]:
net.fit(x_train,
        y_train,
        validation_data=(x_test, y_test),
        epochs=100,
        batch_size=128)

Train on 5170 samples, validate on 1293 samples
Epoch 1/1000
5170/5170 [==============================] - 0s 41us/step - loss: 48.6172 - mae: 6.8874 - val_loss: 40.1395 - val_mae: 6.2546
Epoch 2/1000
5170/5170 [==============================] - 0s 10us/step - loss: 33.8795 - mae: 5.7090 - val_loss: 28.0067 - val_mae: 5.1591
Epoch 3/1000
5170/5170 [==============================] - 0s 9us/step - loss: 23.7816 - mae: 4.6910 - val_loss: 19.8818 - val_mae: 4.2218
Epoch 4/1000
5170/5170 [==============================] - 0s 8us/step - loss: 17.1886 - mae: 3.8445 - val_loss: 14.7001 - val_mae: 3.4754
Epoch 5/1000
5170/5170 [==============================] - 0s 9us/step - loss: 13.0627 - mae: 3.2109 - val_loss: 11.4686 - val_mae: 2.9590
Epoch 6/1000
5170/5170 [==============================] - 0s 9us/step - loss: 10.5008 - mae: 2.7917 - val_loss: 9.4756 - val_mae: 2.6362
Epoch 7/1000
5170/5170 [==============================] - 0s 10us/step - loss: 8.9558 - mae: 2.5233 - val_loss: 8.3200 - va

Epoch 120/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5974 - mae: 0.6060 - val_loss: 0.5605 - val_mae: 0.5798
Epoch 121/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5931 - mae: 0.6042 - val_loss: 0.5598 - val_mae: 0.5809
Epoch 122/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5930 - mae: 0.6040 - val_loss: 0.5592 - val_mae: 0.5802
Epoch 123/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5941 - mae: 0.6045 - val_loss: 0.5590 - val_mae: 0.5781
Epoch 124/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5919 - mae: 0.6043 - val_loss: 0.5592 - val_mae: 0.5774
Epoch 125/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5910 - mae: 0.6032 - val_loss: 0.5576 - val_mae: 0.5804
Epoch 126/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5901 - mae: 0.6023 - val_loss: 0.5575 - val_mae: 0.5810
Epoch 127/1000
5170/5170 [========

5170/5170 [==============================] - 0s 8us/step - loss: 0.5588 - mae: 0.5845 - val_loss: 0.5340 - val_mae: 0.5695
Epoch 239/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5597 - mae: 0.5853 - val_loss: 0.5309 - val_mae: 0.5635
Epoch 240/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5583 - mae: 0.5847 - val_loss: 0.5370 - val_mae: 0.5725
Epoch 241/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5594 - mae: 0.5864 - val_loss: 0.5314 - val_mae: 0.5661
Epoch 242/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5572 - mae: 0.5850 - val_loss: 0.5341 - val_mae: 0.5618
Epoch 243/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5596 - mae: 0.5860 - val_loss: 0.5357 - val_mae: 0.5714
Epoch 244/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5583 - mae: 0.5841 - val_loss: 0.5312 - val_mae: 0.5660
Epoch 245/1000
5170/5170 [=======================

5170/5170 [==============================] - 0s 8us/step - loss: 0.5478 - mae: 0.5792 - val_loss: 0.5219 - val_mae: 0.5578
Epoch 357/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5485 - mae: 0.5793 - val_loss: 0.5216 - val_mae: 0.5597
Epoch 358/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5440 - mae: 0.5775 - val_loss: 0.5217 - val_mae: 0.5581
Epoch 359/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5465 - mae: 0.5778 - val_loss: 0.5214 - val_mae: 0.5597
Epoch 360/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5445 - mae: 0.5765 - val_loss: 0.5302 - val_mae: 0.5699
Epoch 361/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5455 - mae: 0.5777 - val_loss: 0.5213 - val_mae: 0.5600
Epoch 362/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5434 - mae: 0.5763 - val_loss: 0.5217 - val_mae: 0.5579
Epoch 363/1000
5170/5170 [=======================

5170/5170 [==============================] - 0s 8us/step - loss: 0.5375 - mae: 0.5729 - val_loss: 0.5144 - val_mae: 0.5554
Epoch 475/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5358 - mae: 0.5724 - val_loss: 0.5159 - val_mae: 0.5591
Epoch 476/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5360 - mae: 0.5720 - val_loss: 0.5177 - val_mae: 0.5613
Epoch 477/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5384 - mae: 0.5733 - val_loss: 0.5163 - val_mae: 0.5598
Epoch 478/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5370 - mae: 0.5732 - val_loss: 0.5167 - val_mae: 0.5603
Epoch 479/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5418 - mae: 0.5755 - val_loss: 0.5144 - val_mae: 0.5561
Epoch 480/1000
5170/5170 [==============================] - 0s 9us/step - loss: 0.5369 - mae: 0.5734 - val_loss: 0.5153 - val_mae: 0.5588
Epoch 481/1000
5170/5170 [=======================

5170/5170 [==============================] - 0s 8us/step - loss: 0.5295 - mae: 0.5691 - val_loss: 0.5239 - val_mae: 0.5676
Epoch 593/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5329 - mae: 0.5708 - val_loss: 0.5120 - val_mae: 0.5583
Epoch 594/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5310 - mae: 0.5699 - val_loss: 0.5089 - val_mae: 0.5528
Epoch 595/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5302 - mae: 0.5705 - val_loss: 0.5087 - val_mae: 0.5536
Epoch 596/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5283 - mae: 0.5694 - val_loss: 0.5098 - val_mae: 0.5514
Epoch 597/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5289 - mae: 0.5690 - val_loss: 0.5093 - val_mae: 0.5555
Epoch 598/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5300 - mae: 0.5699 - val_loss: 0.5109 - val_mae: 0.5512
Epoch 599/1000
5170/5170 [=======================

5170/5170 [==============================] - 0s 8us/step - loss: 0.5263 - mae: 0.5679 - val_loss: 0.5035 - val_mae: 0.5509
Epoch 711/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5232 - mae: 0.5663 - val_loss: 0.5031 - val_mae: 0.5512
Epoch 712/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5235 - mae: 0.5659 - val_loss: 0.5099 - val_mae: 0.5586
Epoch 713/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5242 - mae: 0.5657 - val_loss: 0.5051 - val_mae: 0.5544
Epoch 714/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5243 - mae: 0.5669 - val_loss: 0.5093 - val_mae: 0.5583
Epoch 715/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5250 - mae: 0.5665 - val_loss: 0.5028 - val_mae: 0.5514
Epoch 716/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5253 - mae: 0.5660 - val_loss: 0.5133 - val_mae: 0.5614
Epoch 717/1000
5170/5170 [=======================

5170/5170 [==============================] - 0s 8us/step - loss: 0.5176 - mae: 0.5631 - val_loss: 0.5038 - val_mae: 0.5553
Epoch 829/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5197 - mae: 0.5647 - val_loss: 0.5021 - val_mae: 0.5477
Epoch 830/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5186 - mae: 0.5644 - val_loss: 0.4991 - val_mae: 0.5494
Epoch 831/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5180 - mae: 0.5634 - val_loss: 0.5013 - val_mae: 0.5474
Epoch 832/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5200 - mae: 0.5649 - val_loss: 0.5014 - val_mae: 0.5531
Epoch 833/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5174 - mae: 0.5637 - val_loss: 0.4986 - val_mae: 0.5497
Epoch 834/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5182 - mae: 0.5641 - val_loss: 0.5058 - val_mae: 0.5475
Epoch 835/1000
5170/5170 [=======================

5170/5170 [==============================] - 0s 8us/step - loss: 0.5147 - mae: 0.5615 - val_loss: 0.4954 - val_mae: 0.5481
Epoch 947/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5128 - mae: 0.5615 - val_loss: 0.4955 - val_mae: 0.5481
Epoch 948/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5160 - mae: 0.5628 - val_loss: 0.4967 - val_mae: 0.5465
Epoch 949/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5141 - mae: 0.5621 - val_loss: 0.4950 - val_mae: 0.5475
Epoch 950/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5136 - mae: 0.5616 - val_loss: 0.5000 - val_mae: 0.5463
Epoch 951/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5162 - mae: 0.5628 - val_loss: 0.4966 - val_mae: 0.5462
Epoch 952/1000
5170/5170 [==============================] - 0s 8us/step - loss: 0.5134 - mae: 0.5615 - val_loss: 0.4960 - val_mae: 0.5467
Epoch 953/1000
5170/5170 [=======================

In [40]:
print("Inputs for the Neural Network:", x_test[0])
prediction = net.predict(x_test[2].reshape([1,13]))
print("Predicted value from the neural network:", prediction)
print("Real value:",y_test[0])
print("Error: ",(prediction[0][0] - y_test[0]))

Inputs for the Neural Network: [0.0 1.0 6.0 0.28 0.34 1.6 0.11900000000000001 33.0 104.0 0.9921 3.19 0.38
 10.2]
Predicted value from the neural network: [[6.192332]]
Real value: 6.0
Error:  0.19233179


In [41]:
test_mse_score, test_mae_score = net.evaluate(x_test, y_test)
print("Mean absolute error:",test_mae_score)
print("Mean squared error:",test_mse_score)

predictions = net.predict(x_test).reshape(y_test.shape)
errors = np.abs(predictions - y_test)

print("Min error:",errors.min())
print("Max error:",errors.max())
print("Standard Deviation:",errors.std())

1293/1293 [==============================] - 0s 11us/step
Mean absolute error: 0.5457080602645874
Mean squared error: 0.49910113008005713
Min error: 0.00024938583
Max error: 2.9095001
Standard Deviation: 0.4486689


In [52]:
#50 best
print(np.sort(errors)[0:50])
#50 worst
print(np.sort(errors)[::-1][0:50])

[0.00024939 0.00024939 0.00320005 0.00433588 0.00468111 0.00533533
 0.00656796 0.00663805 0.00665522 0.00734282 0.00898218 0.01310825
 0.01320219 0.01405621 0.01438522 0.01470709 0.01582193 0.01677084
 0.01969862 0.0199914  0.02100611 0.02442169 0.02468634 0.02473116
 0.02488565 0.0253706  0.02555418 0.02711058 0.02762365 0.02771997
 0.02846909 0.02869844 0.02930975 0.02969503 0.03013372 0.03171206
 0.03322363 0.03325605 0.03352499 0.0345273  0.03476906 0.03497934
 0.03617811 0.036201   0.0366888  0.03678274 0.03725433 0.03926516
 0.03934765 0.0408783 ]
[2.9095001 2.8591514 2.8236637 2.6698823 2.6698823 2.6698823 2.5534964
 2.5534964 2.4414363 2.3231091 2.3069263 2.2845082 2.2711277 2.1843815
 2.0447927 1.999443  1.9946809 1.9295168 1.9266701 1.9133625 1.8960323
 1.8483391 1.836329  1.7975411 1.7868438 1.7868438 1.7778487 1.7625155
 1.748488  1.6881242 1.6770844 1.6709237 1.6674023 1.6662068 1.6609402
 1.651587  1.6445546 1.6445098 1.6243067 1.6065183 1.5963187 1.5826659
 1.5634708 1.5